# 1. Imports & Station List

In [1]:
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
TARGET_STATIONS = [
    "MIT at Mass Ave / Amherst St",
    "Central Square at Mass Ave / Essex St",
    "MIT Pacific St at Purrington St",
    "Harvard Square at Mass Ave / Dunster St",
    "Boylston St at Massachusetts Ave",
    "Charles St at Cambridge St",
    "Forsyth St at Huntington Ave",
    "Boylston St at Fairfield St",
    "Christian Science Plaza - Massachusetts Ave at Westland Ave",
    "MIT Stata Center at Vassar St / Main St",
]


# 2. Load the data

In [3]:
# ---------------------------------------------------------
# 1. LOAD 12 MONTHS OF BLUEBIKES TRIP DATA
# ---------------------------------------------------------

tripdata_path = "2024_data/"   # <-- change to your folder

all_files = glob.glob(tripdata_path + "2024*-bluebikes-tripdata.csv")

print("Files detected:")
for f in all_files:
    print(f)

df_list = []
for filename in all_files:
    print("Loading:", filename)
    df_month = pd.read_csv(filename)
    df_list.append(df_month)

rides = pd.concat(df_list, ignore_index=True)
print("Total rows loaded:", len(rides))

Files detected:


ValueError: No objects to concatenate

In [ ]:

# ---------------------------------------------------------
# 2. LOAD STATION FEATURE DATA (feature.csv)
# ---------------------------------------------------------

station_features = pd.read_csv("feature.csv")   # <-- your uploaded file

# Rename columns to shorter / code-friendly names
station_features = station_features.rename(columns={
    "Station_id": "station_id",
    "Station_name": "station_name",
    "Station latitude": "station_lat",
    "Station longitude": "station_lng",
    "Distance to nearest subway stop (m)": "dist_subway_m",
    "Distance to nearest bus stop (m)": "dist_bus_m",
    "Distance to nearest university (m)": "dist_university_m",
    "Distance to nearest business district": "dist_business",
    "Distance to nearest residential area": "dist_residential",
    "Population density around station": "pop_density",
    "Employment density around station": "emp_density",
    "Number of restaurants/cafes nearby": "restaurant_count",
    "Restaurant/cafes density around station": "restaurant_density",
})

# This file is purely station-level (no time info), so we just keep the static columns.
station_features = station_features[[
    "station_name",
    "station_lat",
    "station_lng",
    "dist_subway_m",
    "dist_bus_m",
    "dist_university_m",
    "dist_business",
    "dist_residential",
    "pop_density",
    "emp_density",
    "restaurant_count",
    "restaurant_density",
]]


## 2.1 Build hourly inflow / outflow per station

In [ ]:
rides["started_at"] = pd.to_datetime(rides["started_at"], format="mixed")
rides["ended_at"] = pd.to_datetime(rides["ended_at"], format="mixed")


# Floor to hour for aggregation
rides["start_hour"] = rides["started_at"].dt.floor("H")
rides["end_hour"] = rides["ended_at"].dt.floor("H")

# Outflow: bikes leaving the station
out_df = (
    rides[rides["start_station_name"].isin(TARGET_STATIONS)]
    .groupby(["start_station_name", "start_hour"])
    .size()
    .reset_index(name="out_count")
    .rename(columns={"start_station_name": "station_name",
                     "start_hour": "hour"})
)

# Inflow: bikes arriving at the station
in_df = (
    rides[rides["end_station_name"].isin(TARGET_STATIONS)]
    .groupby(["end_station_name", "end_hour"])
    .size()
    .reset_index(name="in_count")
    .rename(columns={"end_station_name": "station_name",
                     "end_hour": "hour"})
)

# Merge inflow & outflow into one panel
panel = pd.merge(
    out_df,
    in_df,
    how="outer",
    on=["station_name", "hour"]
)

# Replace NaNs with 0 (no trips that hour)
panel[["in_count", "out_count"]] = panel[["in_count", "out_count"]].fillna(0).astype(int)

panel.head()


## 2.2 Merge with station-level features

In [ ]:
# Keep only the columns you really want from station_features
station_features_clean = station_features[[
    "station_name",
    "station_lat",
    "station_lng",
    "dist_subway_m",
    "dist_bus_m",
    "dist_university_m",
    "dist_business",
    "dist_residential",
    "pop_density",
    "emp_density",
    "restaurant_count",
    "restaurant_density",
]]

# Merge
panel = panel.merge(
    station_features_clean,
    on="station_name",
    how="left"
)

#Time-based features
panel["month"] = panel["hour"].dt.month
panel["day_of_week"] = panel["hour"].dt.dayofweek    # 0 = Monday
panel["hour_of_day"] = panel["hour"].dt.hour
panel["is_weekend"] = panel["day_of_week"].isin([5, 6]).astype(int)  # Sat/Sun



panel.head()


In [ ]:
station_features.columns.tolist()

In [ ]:
# Feature columns
numeric_features = [
    "hour_of_day",
    "day_of_week",
    "month",
    "is_weekend",
    "station_lat",
    "station_lng",
    "dist_subway_m",
    "dist_bus_m",
    "dist_university_m",
    "dist_business",
    "dist_residential",
    "restaurant_count",
]

X = panel[numeric_features].copy()


# 3.Train / test split

In [ ]:
# ---------------------------------------------------------
# 1. Extract target variables
# ---------------------------------------------------------
y_out = panel["out_count"].values   # shape (n_samples,)
y_in  = panel["in_count"].values    # shape (n_samples,)

# ---------------------------------------------------------
# 2. Combine out & in into ONE target matrix
#    Y[:,0] = out_count
#    Y[:,1] = in_count
# ---------------------------------------------------------
Y = np.column_stack([y_out, y_in])   # shape (n_samples, 2)

# ---------------------------------------------------------
# 3. Train-Test Split (same split for both outputs)
# ---------------------------------------------------------
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# ---------------------------------------------------------
# 4. Extract individual outputs after split
# ---------------------------------------------------------
y_train_out = Y_train[:, 0]
y_train_in  = Y_train[:, 1]

y_test_out  = Y_test[:, 0]
y_test_in   = Y_test[:, 1]

print("Shapes:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)


# 4. Build a Poisson regression pipeline

In [ ]:
# Preprocess: scale numeric features
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Poisson regression model
poisson_model = PoissonRegressor(alpha=1.0, max_iter=1000)

# Full pipeline: preprocessing + model
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", poisson_model)
])


# 5. Fit the model and evaluate

In [ ]:
import statsmodels.api as sm
from sklearn.impute import SimpleImputer

# 1. Impute missing values
imputer = SimpleImputer(strategy="median")
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

# 2. Add intercept for statsmodels
X_train_sm = sm.add_constant(X_train_imp)
X_test_sm  = sm.add_constant(X_test_imp)

# 3. Fit Negative Binomial model
model_nb = sm.GLM(
    y_train_in,
    X_train_sm,
    family=sm.families.NegativeBinomial()
).fit()

print(model_nb.summary())

# 4. Predict
y_train_pred = model_nb.predict(X_train_sm)
y_test_pred  = model_nb.predict(X_test_sm)

def print_regression_metrics(split, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"=== {split} ===")
    print(f"MAE:  {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")

# 5. Evaluate
print_regression_metrics("TRAIN", y_train_in, y_train_pred)
print_regression_metrics("TEST", y_test_in, y_test_pred)


# 6. Performance Measurement

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# ---- choose your demand threshold for classification ----
threshold = 4

# convert to binary classes
y_train_true_bin = (y_train_in >= threshold).astype(int)
y_train_pred_bin = (y_train_pred >= threshold).astype(int)

y_test_true_bin  = (y_test_in >= threshold).astype(int)
y_test_pred_bin  = (y_test_pred >= threshold).astype(int)


In [ ]:
from sklearn.metrics import f1_score

best_f1 = 0
best_t = None

for t in range(1, 40):
    pred_bin = (y_test_pred >= t).astype(int)
    f1 = f1_score(y_test_true_bin, pred_bin)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best threshold:", best_t)
print("Best F1:", best_f1)


In [ ]:
def print_classification_metrics(split, y_true, y_pred):
    print(f"\n=== {split} — Classification Metrics ===")
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:   ", recall_score(y_true, y_pred))
    print("F1 Score: ", f1_score(y_true, y_pred))

print_classification_metrics("TRAIN", y_train_true_bin, y_train_pred_bin)
print_classification_metrics("TEST", y_test_true_bin, y_test_pred_bin)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion(split, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Pred Low", "Pred High"],
                yticklabels=["True Low", "True High"])
    plt.title(f"{split} Confusion Matrix")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

plot_confusion("TRAIN", y_train_true_bin, y_train_pred_bin)
plot_confusion("TEST", y_test_true_bin, y_test_pred_bin)
